In [34]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [47]:
df_1 = pd.read_csv('round-2-island-data-bottle/prices_round_2_day_-1.csv',delimiter=';')
df_2 = pd.read_csv('round-2-island-data-bottle/prices_round_2_day_0.csv',delimiter=';')
df_3 = pd.read_csv('round-2-island-data-bottle/prices_round_2_day_1.csv',delimiter=';')

In [48]:
for df in [df_1, df_2, df_3]:
    # List of specified columns
    columns = ['ORCHIDS', 'TRANSPORT_FEES', 'EXPORT_TARIFF', 'IMPORT_TARIFF', 'SUNLIGHT', 'HUMIDITY']
    
    # Calculate the absolute difference between consecutive rows for each specified column
    for column in columns:
        df[f'diff_{column}'] = df[column].diff()
    
    # Your df_1 will now have new columns with names 'abs_diff_ORCHIDS', 'abs_diff_TRANSPORT_FEES', etc.,
    # containing the absolute differences for each respective column.
    df['target'] = (df['ORCHIDS'].diff() > 0).astype(int).shift(-1)


In [49]:
df_1 = df_1.iloc[1:]
df_2 = df_2.iloc[1:]
df_3 = df_3.iloc[1:]

In [51]:
df_1['target'].value_counts()

target
0.0    6006
1.0    3993
Name: count, dtype: int64

In [52]:
df = pd.concat([df_1, df_2, df_3])

In [55]:
# Assuming 'df' is your DataFrame
df = df.dropna()

# After this operation, 'df' will only contain rows without any NaN values.
df.reset_index()

,index,timestamp,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY,diff_ORCHIDS,diff_TRANSPORT_FEES,diff_EXPORT_TARIFF,diff_IMPORT_TARIFF,diff_SUNLIGHT,diff_HUMIDITY,target
0,1,100,1201.75,1.5,9.5,-2.0,2499.4197,79.00410,-1,1.75,0.0,-1.0,0.0,-0.5803,0.00410,0.0
1,2,200,1201.75,1.5,9.5,-2.0,2498.8457,79.00821,-1,0.00,0.0,0.0,0.0,-0.5740,0.00411,0.0
2,3,300,1201.75,1.5,9.5,-2.0,2498.2780,79.01234,-1,0.00,0.0,0.0,0.0,-0.5677,0.00413,0.0
3,4,400,1201.75,1.5,9.5,-2.0,2497.7166,79.01649,-1,0.00,0.0,0.0,0.0,-0.5614,0.00415,0.0
4,5,500,1200.25,1.5,9.5,-2.0,2497.1614,79.02066,-1,-1.50,0.0,0.0,0.0,-0.5552,0.00417,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,9995,999500,1038.75,0.9,9.5,-5.0,2299.9330,69.91952,1,-1.00,0.0,0.0,0.0,0.0092,0.01591,0.0
29993,9996,999600,1036.25,0.9,9.5,-5.0,2299.9436,69.93549,1,-2.50,0.0,0.0,0.0,0.0106,0.01597,0.0
29994,9997,999700,1036.25,0.9,9.5,-5.0,2299.9556,69.95153,1,0.00,0.0,0.0,0.0,0.0120,0.01604,0.0
29995,9998,999800,1036.25,0.9,9.5,-5.0,2299.9690,69.96762,1,0.00,0.0,0.0,0.0,0.0134,0.01609,0.0


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Load your datasets and combine them
# For demonstration, I will use the iris dataset
X = df[['ORCHIDS', 'TRANSPORT_FEES', 'EXPORT_TARIFF', 'IMPORT_TARIFF', 'SUNLIGHT', 'HUMIDITY',"diff_ORCHIDS","diff_TRANSPORT_FEES","diff_EXPORT_TARIFF","diff_IMPORT_TARIFF","diff_SUNLIGHT","diff_HUMIDITY"] ]
y = df.target

# Define the parameter grid you want to search over
param_grid = {
    'n_estimators': [10, 20, 30,50, 70,100, 200, 300],
    'max_depth': [3,5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    #maybe you can add learning rate
    # Add other parameters if needed
}

# Create a RandomForestClassifier instance
rf_clf = RandomForestClassifier(random_state=42)

# Create a GridSearchCV instance with the RandomForestClassifier
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the GridSearchCV instance to your data
grid_search.fit(X, y)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters found: ", best_params)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
